<a href="https://colab.research.google.com/github/shubhamkambleX/Auto-Annotation-/blob/main/Auto_Annotation_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision

In [2]:
import warnings
warnings.simplefilter("ignore")
import os
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw

In [ ]:
# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define a function for image annotation
def annotate_image(image_path, output_path='annotated_images', confidence_threshold=0.5):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Process each image in the directory
    for filename in os.listdir(image_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Load the image
            img_path = os.path.join(image_path, filename)
            img = Image.open(img_path).convert("RGB")

            # Preprocess the image
            transform = T.Compose([T.ToTensor()])
            input_img = transform(img).unsqueeze(0)

            # Perform inference
            with torch.no_grad():
                prediction = model(input_img)

            # Draw bounding boxes on the image
            draw = ImageDraw.Draw(img)
            boxes = prediction[0]['boxes'].cpu().numpy()
            scores = prediction[0]['scores'].cpu().numpy()

            # Create a list to store coordinates
            coordinates = []

            for box, score in zip(boxes, scores):
                if score > confidence_threshold:
                    draw.rectangle(box, outline="red", width=3)
                    coordinates.append(box.tolist())

            # Save annotated image
            output_img_path = os.path.join(output_path, f'annotated_{filename}')
            img.save(output_img_path)

            # Save coordinates to a text file
            output_txt_path = os.path.join(output_path, f'coordinates_{filename}.txt')
            with open(output_txt_path, 'w') as f:
                for coord in coordinates:
                    f.write(','.join(map(str, coord)) + '\n')

# Test the annotation function on a directory of images
annotate_image('/content/input_data', output_path='/content/output_data', confidence_threshold=0.5)


In [3]:
# import os
# from PIL import Image, ImageDraw
# import torch
# from torchvision.models.detection import fasterrcnn_resnet50_fpn
# from torchvision.transforms import functional as T
# torchvision.transforms

# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define a function for image annotation
def annotate_image(image_path, output_path='annotated_images', confidence_threshold=0.5):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Process each image in the directory
    for filename in os.listdir(image_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Load the image
            img_path = os.path.join(image_path, filename)
            img = Image.open(img_path).convert("RGB")

            # Preprocess the image
            transform = T.Compose([T.ToTensor()])
            input_img = transform(img).unsqueeze(0)

            # Perform inference
            with torch.no_grad():
                prediction = model(input_img)

            # Draw bounding boxes on the image
            draw = ImageDraw.Draw(img)
            boxes = prediction[0]['boxes'].cpu().numpy()
            scores = prediction[0]['scores'].cpu().numpy()
            labels = prediction[0]['labels'].cpu().numpy()

            # Create a list to store coordinates and labels
            annotations = []

            for box, score, label in zip(boxes, scores, labels):
                if score > confidence_threshold:
                    draw.rectangle(box, outline="red", width=3)
                    coordinates = box.tolist()
                    label_info = {'label': label, 'coordinates': coordinates}
                    annotations.append(label_info)

            # Save annotated image
            output_img_path = os.path.join(output_path, f'annotated_{filename}')
            img.save(output_img_path)

            # Save coordinates and labels to a text file
            output_txt_path = os.path.join(output_path, f'annotations_{filename}.txt')
            with open(output_txt_path, 'w') as f:
                for annotation in annotations:
                    label_str = f"label: {annotation['label']}"
                    coord_str = f"coordinates: {','.join(map(str, annotation['coordinates']))}"
                    f.write(f"{label_str}, {coord_str}\n")

# Test the annotation function on a directory of images
annotate_image('/content/input_data', output_path='/content/output_data', confidence_threshold=0.5)


In [6]:
# import os
# from PIL import Image, ImageDraw
# import torch
# from torchvision.models.detection import fasterrcnn_resnet50_fpn
# from torchvision.transforms import functional as T

# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define a function for image annotation
def annotate_image(image_path, output_path='annotated_images', confidence_threshold=0.5,dynamic_val = [0,1,2]):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Process each image in the directory
    for filename in os.listdir(image_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Load the image
            img_path = os.path.join(image_path, filename)
            img = Image.open(img_path).convert("RGB")

            # Preprocess the image
            transform = T.Compose([T.ToTensor()])
            input_img = transform(img).unsqueeze(0)

            # Perform inference
            with torch.no_grad():
                prediction = model(input_img)

            # Extract bounding boxes, scores, and labels
            draw = ImageDraw.Draw(img)
            boxes = prediction[0]['boxes'].cpu().numpy()
            scores = prediction[0]['scores'].cpu().numpy()
            labels = prediction[0]['labels'].cpu().numpy()

            # Create a list to store annotations
            annotations = []

            for box, score, label in zip(boxes, scores, labels):
                if score > confidence_threshold:
                    draw.rectangle(box, outline="red", width=3)
                    # Append label and coordinates to the list
                    lab_name = dynamic_val[label] if dynamic_val and label < len(dynamic_val) else 0
                    annotation_str = f"{lab_name} {' '.join(map(str, box.tolist()))}"
                    annotations.append(annotation_str)

            # Save annotated image
            output_img_path = os.path.join(output_path, f'annotated_{filename}')
            img.save(output_img_path)

            # Save annotations to a text file
            output_txt_path = os.path.join(output_path, f'annotations_{filename}.txt')
            with open(output_txt_path, 'w') as f:
                for annotation in annotations:
                    f.write(annotation + '\n')

# Test the annotation function on a directory of images
annotate_image('/content/input_data', output_path='/content/output_data', confidence_threshold=0.5)
